#### Notebook contents 

* BERT- Bidirectional Encoder Representations from Transformers is a transformer-based machine learning technique for natural language processing pre-training developed by Google. BERT was created and published in 2018 by Jacob Devlin and his colleagues from Google

* Installing & importing packages 

* Cloning github repo




In [ ]:
import numpy as np
import pandas as pd
import ktrain
from ktrain import text

In [ ]:
import tensorflow as tf

In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.3 MB 2.0 MB/s 
     |████████████████████████████████| 22.3 MB 55.9 MB/s 
     |████████████████████████████████| 981 kB 35.2 MB/s 
     |████████████████████████████████| 263 kB 49.5 MB/s 
     |████████████████████████████████| 2.8 MB 35.3 MB/s 
     |████████████████████████████████| 1.2 MB 35.4 MB/s 
     |████████████████████████████████| 468 kB 45.3 MB/s 
     |████████████████████████████████| 67 kB 4.6 MB/s 
     |████████████████████████████████| 3.3 MB 35.8 MB/s 
     |████████████████████████████████| 895 kB 47.6 MB/s 
     |████████████████████████████████| 596 kB 36.2 MB/s 
  Created wheel for ktrain: filename=ktrain-0.29.2-py3-none-any.whl size=25294171 sha256=6b6d5b900d7f5c628b5b53f5d29d58165aceec158034f78315c0ef02e6c782f4
  Stored in directory: /root/.cache/pip/wheels/db/b1/ad/94fe0ceeaa8ed0319e74773cba99b6aa6192998b1035adc295
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9931 sha256=787897fb0e31ec5

In [ ]:
tf.__version__

'2.8.0'

In [ ]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [ ]:
data_train = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx', dtype = str)

In [ ]:
data_test = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx',dtype = str)

In [ ]:
data_train.tail(7)

,Reviews,Sentiment
24993,"""Pushing Daisies"" for sure is one of the best ...",pos
24994,This movie has to be my favorite of all time. ...,pos
24995,Everyone plays their part pretty well in this ...,pos
24996,It happened with Assault on Prescient 13 in 20...,neg
24997,My God. This movie was awful. I can't complain...,neg
24998,"When I first popped in Happy Birthday to Me, I...",neg
24999,"So why does this show suck? Unfortunately, tha...",neg


In [ ]:
data_test.head(7)

,Reviews,Sentiment
0,Who would have thought that a movie about a ma...,pos
1,After realizing what is going on around us ......,pos
2,I grew up watching the original Disney Cindere...,neg
3,David Mamet wrote the screenplay and made his ...,pos
4,"Admittedly, I didn't have high expectations of...",neg
5,OK - you want to test somebody on how comforta...,pos
6,The script is nice.Though the casting is absol...,neg


In [ ]:
data_train.shape

(25000, 2)

In [ ]:
data_test.shape

(25000, 2)

In [ ]:
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_df(train_df = data_train, text_column= 'Reviews',
                                                                   label_columns= 'Sentiment', val_df= data_test,
                                                                   maxlen= 500, preprocess_mode= 'bert')


['neg', 'pos']
   neg  pos
0  1.0  0.0
1  1.0  0.0
2  1.0  0.0
3  1.0  0.0
4  1.0  0.0
['neg', 'pos']
   neg  pos
0  0.0  1.0
1  0.0  1.0
2  1.0  0.0
3  0.0  1.0
4  1.0  0.0
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
model = text.text_classifier(name = 'bert', train_data = (X_train, y_train), preproc = preproc)

Is Multi-Label? False
maxlen is 500
done.


In [ ]:
learner = ktrain.get_learner(model = model, train_data = (X_train, y_train), val_data = (X_test, y_test), batch_size = 6)

In [ ]:
learner.fit_onecycle(lr=2e-5, epochs=1)



begin training using onecycle policy with max lr of 2e-05...
4167/4167 [==============================] - 9418s 2s/step - loss: 0.2541 - accuracy: 0.8964 - val_loss: 0.1129 - val_accuracy: 0.9630


In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
data = ['this was a well put together film. great acting', 
        'would rewatch this movie again and again',
        'do no waste time seeing this movie. I would rather watch paint dry']

In [ ]:
predictor.predict(data)

['pos', 'pos', 'neg']

In [ ]:
predictor.predict(data, return_proba = True)

array([[0.00419933, 0.9958007 ],
       [0.05147934, 0.9485207 ],
       [0.70741785, 0.29258215]], dtype=float32)

In [ ]:
predictor.get_classes()

['neg', 'pos']

In [ ]:
predictor.save('/content/bert')

In [ ]:
!zip -r /content/bert.zip /content/bert 

  adding: content/bert/ (stored 0%)
  adding: content/bert/tf_model.preproc (deflated 52%)
  adding: content/bert/tf_model.h5 (deflated 11%)


In [ ]:
predictor_load = ktrain.load_predictor('/content/bert')

In [ ]:
predictor_load.get_classes()

['neg', 'pos']

In [ ]:
predictor_load.predict(data)

['pos', 'pos', 'neg']